In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
#@title cd
%pwd
%cd /content/drive/MyDrive/afeka/Project/code/PTEEnet/PTEEnet

/content/drive/MyDrive/afeka/Project/code/PTEEnet/PTEEnet


In [36]:
#@title install packages
%pip install chainer
%pip install -Uqq ipdb

In [37]:
#@title read gpu details
import chainer
#chainer.print_runtime_info()

print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

GPU availability: True
cuDNN availablility: True
Thu Jan 13 14:48:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    35W / 250W |   1023MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                       

In [38]:
#@title imports
"""
high level support for doing this and that.
"""
from __future__ import print_function
import time
import csv
import numpy as np
import torch
import torch.nn.functional as F
from scipy import stats
from init import initializer
from eenet import EENet
from custom_eenet import CustomEENet
from gatednet import GatedNet
import matplotlib.pylab as plt
import loss_functions
import utils
import config
import gc
import sys
import math
import os
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms.functional as tf
import matplotlib.pyplot as plt
import ipdb

import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import joblib as jobl
from collections import Counter
from matplotlib import pyplot
from numpy import where
from imblearn.over_sampling import RandomOverSampler 
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor

%matplotlib inline


In [39]:
#@title plot_ee_histogram
def plot_ee_histogram(args, val_confs, type_loader, dir):
    %matplotlib inline
    x_pos = np.arange(args.num_ee+1)
    #print(x_pos)
    print(val_confs)
    bar = plt.bar(x_pos, val_confs, align='center')
    plt.xlabel('Exit number')
    plt.ylabel('# of Samples in Exit')
    plt.title(args.dataset + ' samples distribution in exits using ' + args.model + 
              ' model\n' + 'SNR=' + str(args.noise_snr) + 'dB')
    plt.ylim(0, max(val_confs))
    plt.savefig(dir + '/ee_hist_' + type_loader + '.' + str(args.loss_threshold) + '.png')
    plt.show()
    

In [40]:
#@title plot_noise_exits
import plotly.graph_objects as go
import matplotlib

#@title plot_noise_exits
def plot_noise_exits_box(args, type_loader):
    %matplotlib inline
    x_pos = np.arange(args.num_noise_levels)

    for thresh in range(1, 13, 1):
        loss_threshold = thresh/10
        data_file = args.relative_losses_dir + '/noise_exits_' + type_loader + '.' + str(loss_threshold) + '.pt'
        noise_exits = torch.load(data_file)
        #noise_exits = np.array(noise_exits)

        #ipdb.set_trace(context=6)
        mean=np.mean(noise_exits,axis=1)
        print(mean)

        fig, ax = plt.subplots()#figure(figsize =(10, 7))
    
        # Creating axes instance
        ax.set_title(args.dataset + ' samples distribution in exits using ' + args.model + ' with threshold=' + str(loss_threshold))
        ax.set_xlabel('noise index')
        ax.set_ylabel('# exits')
        
        # Creating plot
        bp = ax.boxplot(noise_exits)
        
        # show plot
        plt.show()
        plt.savefig(args.relative_losses_dir + '/bp_noises_exits_' + '.' + str(loss_threshold) + '.png')
        
def plot_noise_exits_scatter(args, type_loader):
    %matplotlib inline
    #x_pos = np.arange(args.num_noise_levels)
    noise_levels = np.arange(args.min_noise_snr, args.max_noise_snr,
        (args.max_noise_snr - args.min_noise_snr)/args.num_noise_levels) 

    x = noise_levels
    selected_losses = np.array([0.3, 0.5, 0.7, 0.9])
    colors = ['green', 'black', 'red', 'blue']
    fig = go.Figure()

    #for thresh in selected_losses:
    for idx, thresh in enumerate(selected_losses):
        loss_threshold = thresh#/10
        data_file = args.relative_losses_dir + '/noise_exits_' + type_loader + '.' + str(loss_threshold) + '.pt'
        data = torch.load(data_file)
        #noises_data.append(data)

        #mean=np.median(data,axis=1)
        mean=np.mean(data,axis=1)
        std=np.std(data,axis=1)
        q1=np.quantile(data,q=0.25,axis=1)
        q3=np.quantile(data,q=0.75,axis=1)
        """
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(x=x, y=q3,
                                mode='lines',
                                line=dict(color='red',width =0.1),
                                name='upper bound'))
        """
        fig.add_trace(go.Scatter(x=x, y=mean,
                                mode='lines',
                                line=dict(color=colors[idx]),
                                #fill='tonexty',
                                name='threshold ' + str(loss_threshold)))
        """
        fig.add_trace(go.Scatter(x=x, y=q1,
                                mode='lines',
                                line=dict(color='blue', width =0.1),
                                fill='tonexty',
                                name='lower bound'))
        """
        #ipdb.set_trace(context=6)
    fig.update_layout(
        title=args.dataset + ' samples distribution in exits using ' + args.model,# + ' with threshold=' + str(loss_threshold),
        xaxis_title="noise level in SNR (dB)",
        yaxis_title="# exit",
        xaxis = dict(
            tickmode = 'array',
            tickvals = noise_levels,
        ),
        width=700,
        height=500)

    #plt.show()
    fig.show()
    plt.savefig(args.relative_losses_dir + '/bp_noises_exits_' + '.' + str(args.loss_threshold) + '.png')

In [41]:
#@title exit_flow_analysis
def exit_flow_analysis(args, model, val_loader):
    """examine the model output.
    Arguments are
    * args:         command line arguments entered by user.
    * model:        convolutional neural network model.
    * train_loader: train data loader.
    This examines the outputs of already trained model.
    """
    model.eval()
    val_confs_true = []
    val_confs_false = []
    vals_len_true = np.zeros(args.num_ee+1)
    vals_len_false = np.zeros(args.num_ee+1)
    for i in range(args.num_ee+1):
        val_confs_true.append([])
        val_confs_false.append([])
    
    #val_confs[0].append('aa2')
    #val_confs = np.empty((args.num_ee, 0), float)#np.zeros(len(val_loader.dataset))
    #i = 0
    """
    print(args.results_dir+'/pred_vs_conf.csv')
    experiment = open(args.results_dir+'/pred_vs_conf.csv', 'w', newline='')
    recorder = csv.writer(experiment, delimiter=',')
    recorder.writerow(['target',
                       'start_pred_seq',
                       'start_conf_seq',
                       'start_exit_seq',
                       'actual_pred',
                       'actual_conf',
                       'actual_exit'])
    """
    
    print('add noise w/ SNR=' + str(args.testing_snr) + 'dB')
    
    with torch.no_grad():
        for data, target in val_loader:
            data = add_noise(args, data)
            
            data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
            pred, conf, cost, idx = model(data)
            loss = F.nll_loss(pred.log(), target)

            pred = pred.max(1, keepdim=True)[1].item()
            conf = conf.item()
            #print(loss)
            #conf = torch.max(confs).item()
            #i += 1
            #confs = [c.item() for c in confs]
            loss = loss.item()
            target = target.item()
            if target == pred:
                val_confs_true[idx].append(conf)
            else:
                val_confs_false[idx].append(conf)

            #val_confs[idx].append(conf)

    total_correct = 0
    for i in range(args.num_ee+1):
        total_correct += len(val_confs_true[i])
        if len(val_confs_true[i]) > 1:
            filename = 'test_true_confs_hist_ee' + str(i)
            vals_len_true[i] = len(val_confs_true[i])
            utils.plot_histogram(args, val_confs_true[i], 'conf', 100, filename, saveplot=True)
        if len(val_confs_false[i]) > 1:
            filename = 'test_false_confs_hist_ee' + str(i)
            vals_len_false[i] = len(val_confs_false[i])
            utils.plot_histogram(args, val_confs_false[i], 'conf', 100, filename, saveplot=True)
    flat_list_true = [item for sublist in val_confs_true for item in sublist]
    flat_list_false = [item for sublist in val_confs_false for item in sublist]
    
    utils.plot_histogram(args, flat_list_true, 'conf true', 100, 'global_conf_true', saveplot=True)
    utils.plot_histogram(args, flat_list_false, 'conf false', 100, 'global_conf_false', saveplot=True)
    print('total currect: ', total_correct)
    print('total acc : ', total_correct * 100 /10000)
    plot_ee_histogram(args, vals_len_true)
    plot_ee_histogram(args, vals_len_false)
    return

In [42]:
#@title Train
def train(args, model, train_loader, optimizer):
    """train the model.

    Arguments are
    * args:         command line arguments entered by user.
    * model:        convolutional neural network model.
    * train_loader: train data loader.
    * optimizer:    optimize the model during training.
    * epoch:        epoch number.

    This trains the model and prints the results of each epochs.
    """
    losses = []
    pred_losses = []
    cost_losses = []
    model.train()
    
    # actual training starts
    #for batch_id, ((data, noise), target) in enumerate(train_loader):
    for batch_id, (data, target) in enumerate(train_loader):
        # fetch the current batch data
        data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
        exit_tag = None
        optimizer.zero_grad()
        cum_loss = 0
        cost = []

        # training settings for EENet based models
        if isinstance(model, (CustomEENet, EENet)):
            pred, conf = model(data)
            #ipdb.set_trace(context=6)
            cost.append(1)#torch.tensor(1.0).to(args.device))
            
            if args.use_main_targets:
                _, target = torch.max(pred[args.num_ee], 1)
                         
            cum_loss, pred_loss, cost_loss = loss_functions.loss(args, exit_tag, pred, target, conf, cost)
            
        # training settings for other models
        else:
            pred = model(data)
            cum_loss = F.cross_entropy(pred, target)

        losses.append(float(cum_loss))
        pred_losses.append(float(pred_loss))
        cost_losses.append(float(cost_loss))
        cum_loss.backward()
        optimizer.step()
        """
        for obj in gc.get_objects():
            try:
                if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                    print(type(obj), obj.size())
            except:
                pass
        """
        #print(model.exits[1].classifier[0].weight)

        # update the exit tags of inputs
    #print(pred_loss)
    #print('target')
    #print(target)
    
    # print the training results of epoch
    result = {'train_loss': round(np.mean(losses), 4),
              'train_loss_sem': round(stats.sem(losses), 2),
              'pred_loss': round(np.mean(pred_losses), 4),
              'pred_loss_sem': round(stats.sem(pred_losses), 2),
              'cost_loss': round(np.mean(cost_losses), 4),
              'cost_loss_sem': round(stats.sem(cost_losses), 2)}

    print('Train avg loss: {:.4f}'.format(result['train_loss']))
    
    return result

In [43]:
#@title Validate

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def validate(args, model, val_loader):
    batch = {'time':[], 'cost':[], 'flop':[], 'acc':[], 'val_loss':[]}
    exit_points = [0]*(args.num_ee+1)
    # switch to evaluate mode
    model.eval()
    main_ee_losses = []
        
    with torch.no_grad():
        
        for batch_id, (data, target) in enumerate(val_loader):
            batch_size = target.shape[0]        
            data, target = data.to(args.device), target.to(args.device, dtype=torch.int64)
            # compute output
            start = time.process_time()

            # results of EENet based models
            if isinstance(model, (EENet, CustomEENet)):                
                preds, confs = model(data)
                
                if args.use_main_targets:
                    _, target = torch.max(preds[args.num_ee], 1)                
        
                if mode == Mode.train_main:
                    pred = preds[args.num_ee]
                    cost = 1
                    exit_points[args.num_ee] += batch_size
                    flop = model.complexity[-1][0]
                    pred_loss = F.nll_loss(pred.log(), target)
                    main_ee_losses.append(float(main_ee_loss))
                    
                elif mode == Mode.train_ee:
                    losses = []
                    costs = []
                    flops = []

                    pred_losses = torch.empty((args.num_ee + 1, batch_size), dtype=torch.float)
                    pred = torch.zeros((batch_size, preds[0].shape[1]), dtype=torch.float, device = 'cuda')
                    
                    for i in range(args.num_ee+1):
                        cost = model.complexity[i][0]/model.complexity[-1][0]
                        pred_losses[i] = F.nll_loss(preds[i].log(), target, reduction='none') +\
                                args.lambda_coef * cost
                    main_ee_loss = F.nll_loss(preds[args.num_ee].log(), target)
                    main_ee_losses.append(float(main_ee_loss))
                    
                    for j in range(batch_size):
                        best_exit = args.num_ee
                        for i in range(args.num_ee):    
                            if pred_losses[i][j] < args.loss_threshold:
                                best_exit = i
                                break
                        
                        #ipdb.set_trace(context=3)
                        exit_points[best_exit] += 1
                        best_pred_loss = pred_losses[best_exit][j]
                        losses.append(float(best_pred_loss))
                        cost = model.complexity[best_exit][0]/model.complexity[-1][0] 
                        flop = model.complexity[best_exit][0]
                        flops.append(float(flop))
                        costs.append(float(cost))
                        pred[j] = preds[best_exit][j]
                    
                    #ipdb.set_trace(context=3)
                    cost = round(np.mean(costs), 4)
                    pred_loss = round(np.mean(losses), 4)
                    flop = round(np.mean(flops), 4)
                    #ipdb.set_trace(context=3)                    
                    losses = None
                    costs = None
                    flops = None                    
                else:
                    print('Unknown mode')
                    
            elapsed_time = time.process_time() - start
            prec1 = accuracy(pred.float().data, target)[0]
            pred = None
                
            batch['acc'].append(float(prec1))
            batch['time'].append(elapsed_time)
            batch['cost'].append(cost*100.)
            batch['flop'].append(flop)
            batch['val_loss'].append(float(pred_loss))
            
    print('main ee avg loss: ', round(np.mean(main_ee_losses), 4))
    utils.print_validation(args, batch, exit_points)
    
    result = {}
    for key, value in batch.items():
        result[key] = round(np.mean(value), 4)
        result[key+'_sem'] = round(stats.sem(value), 2)
    
    result['exit_points'] = exit_points
    return result


In [44]:
#@title run()
def run(model, optimizer, lr_scheduler, args, train_loader, test_loader):
    # best = {}
    # best_epoch = 0
    
    print('Running for {:5d} epochs'.format(args.epochs))
    try:
        if not args.no_tensorboard:
            writer = SummaryWriter('../runs/train_eenet_experiment_1')
        torch.cuda.empty_cache()
        for epoch in range(args.start_epoch, args.epochs + 1):
            print('{:3d}:'.format(epoch), end='')

            # two-stage training uses the loss version-1 after training for 25 epochs
            if args.two_stage and epoch > 25:
                args.loss_func = "v1"
            
            result = {'epoch':epoch}
            
            result.update(train(args, model, train_loader, optimizer))
            if not args.no_tensorboard:
                writer.add_scalar("Train/loss", result['train_loss'], epoch)

            # use adaptive learning rate
            if args.adaptive_lr:
                lr_scheduler.step()

            # validate and keep history at each log interval
            if epoch % args.log_interval == 0:
                result.update(validate(args, model, test_loader))
                utils.save_history(args, result)
                if not args.no_tensorboard:
                    writer.add_scalar("Val/loss", result['val_loss'], epoch)
                    writer.add_scalars('Val/acc_cost', {
                        'acc': result['acc'],
                        'cost': result['cost']
                    }, epoch)
            if not args.no_tensorboard:    
                writer.flush()
            # save model parameters
            if not args.no_save_model:
                utils.save_model(args, model, epoch)
    except KeyboardInterrupt:
        utils.close_history(args)
        utils.plot_history(args)
        if not args.no_tensorboard:
            writer.close()
        sys.exit()
    # print the best validation result
    best_epoch = utils.close_history(args)

    # save the model giving the best validation results as a final model
    if args.save_best:
        utils.save_model(args, model, best_epoch, True)
    utils.plot_history(args)
    if not args.no_tensorboard:
        writer.close()


In [45]:
#@title enable_branches_training_only
def enable_branches_training_only(model, args):
    if args.model == 'eenet8':
        model.set_ee_disable(False)
        model.initblock.requires_grad_(False)
        model.basicblock1.requires_grad_(False)
        model.basicblock2.requires_grad_(False)
        model.basicblock3.requires_grad_(False)
        model.finalblock.requires_grad_(False)
        model.classifier.requires_grad_(False)
        model.conv2d_6.requires_grad_(False)
        model.conv2d_9.requires_grad_(False)
    else:
        model.set_ee_disable(False)
        for idx, exitblock in enumerate(model.exits):
            model.stages[idx].requires_grad_(False)
            for param in model.exits.parameters():
                param.requires_grad = True
            #model.exits.requires_grad(True)

        model.stages[-1].requires_grad_(False)
        model.classifier.requires_grad_(False)
        model.confidence.requires_grad_(False)
        
        #params = model_post.state_dict()
        #print(params)
        
        """
        ipdb.set_trace(context=3)
        for name, p in model.named_parameters():
            if p.requires_grad:
                print(name, p.requires_grad)
        """
                

In [46]:
#@title load data
from torch.utils.data import Dataset, TensorDataset, DataLoader

class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        #ipdb.set_trace(context=6)
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]
        
        return x, y

    def __len__(self):
        return self.tensors[0].size(0)
        
#def load_data(args):
#    return utils.load_custom_dataset(args)

def load_data(args):
    serialized_data_dir = 'serialized_data'
    
    data_file_train = serialized_data_dir + '/datas_train' + '.pt'
    data_train = torch.load(data_file_train)
    print('loaded data train at: ', data_file_train)
    
    data_file_test = serialized_data_dir + '/datas_test' + '.pt'
    data_test = torch.load(data_file_test)
    print('loaded data test at: ', data_file_test)
    
    gt_file_train = serialized_data_dir + '/targets_train' + '.pt'
    target_train = torch.load(gt_file_train)
    print('loaded exits ground truths train at: ', gt_file_train)
    
    gt_file_test = serialized_data_dir + '/targets_test' + '.pt'
    target_test = torch.load(gt_file_test)
    print('loaded exits ground truths test at: ', gt_file_test)

    data_train, data_test, target_train, target_test = train_test_split(data_train,\
                                                                        target_train, test_size=0.33, random_state=42)
    
    data_train, target_train = torch.from_numpy(data_train), torch.from_numpy(target_train)    
    data_test, target_test = torch.from_numpy(data_test), torch.from_numpy(target_test)
    
    train_set = CustomTensorDataset(tensors=(data_train, target_train))    
    validation_set = CustomTensorDataset(tensors=(data_test, target_test))
    loader_train = torch.utils.data.DataLoader(train_set, batch_size=args.batch_size, shuffle=args.shuffle_train)
    loader_val = torch.utils.data.DataLoader(validation_set, batch_size=args.test_batch, shuffle=args.shuffle_test)

    return loader_train, loader_val

In [47]:
#@title main

"""Main function of the program.

The function loads the dataset and calls training and validation functions.
"""
%load_ext autoreload
%autoreload 2

import importlib
importlib.reload(config)
importlib.reload(loss_functions)
importlib.reload(utils)
#from eenet import EENet
#from gatednet import GatedNet, GatedNetS
from enum import Enum

class Mode(Enum):
    train_main = 0
    train_ee = 1
    generate_exits_gt = 2 
    train_gating = 3
    generate_relative_loss = 4 
    plot_relative_loss = 5
    calc_relative_time = 6

def main(mode: Mode):
    print(mode.value)
    args = config.args_global
    args += config.argu[mode.value]
    
    print(args)
    %pwd
    model, optimizer, lr_scheduler, args = initializer(args)
    
    if mode == Mode.train_main:
        train_loader, test_loader, exit_tags, trainset, testset = load_data(args)
        model.set_ee_disable(True)
        print('Disabled EE branches')
        run(model, optimizer, lr_scheduler, args, train_loader, test_loader, exit_tags)
    elif mode == Mode.train_ee:
        #train_loader, test_loader, exit_tags, trainset, testset = load_data(args)
        train_loader, test_loader = load_data(args)
        enable_branches_training_only(model, args)
        print('Enabled EE branches')
        print('loss threshold: ', args.loss_threshold)
        run(model, optimizer, lr_scheduler, args, train_loader, test_loader)
    
    print('Finished')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
mode = Mode.train_ee
main(mode)

1
['--dataset', 'cifar10', '--model', 'eenet110', '--num-ee', '10', '--filters', '4', '--exit-type', 'conv2', '--distribution', 'fine', '--min-noise-snr', '-5', '--max-noise-snr', '30', '--num-noise-levels', '5', '--validate', '--epochs', '50', '--lambda-coef', '0.0', '--optimizer', 'Adam', '--lr', '0.001', '--weight-decay', '0.0001', '--load-model', 'models/cifar10/eenet110/UT/clean/after_main_training/model.pt', '--use-main-targets', '--loss-func', 'v0', '--no-tensorboard', '--clear-dirs', '--batch-size', '128', '--test-batch', '128', '--log-interval', '1', '--loss-threshold', '0.2', '--shuffle-train']
use cuda:  True  device:  cuda
empty model loaded at:  models/cifar10/eenet110/UT/clean/after_main_training/model.pt
ee-block-0: flops=19.78 MMac, params=19.34 k, cost-rate=0.08
ee-block-1: flops=34.23 MMac, params=33.35 k, cost-rate=0.13
ee-block-2: flops=43.86 MMac, params=42.7 k, cost-rate=0.17
ee-block-3: flops=53.49 MMac, params=52.04 k, cost-rate=0.21
ee-block-4: flops=63.13 MMac